In [1]:
using LinearAlgebra

include("../src/tebd.jl")
using .TimeEvolvingBlockDecimation
using .TimeEvolvingBlockDecimation.MatrixProductState

Let's initialize a 12 qubit state in the $|0\cdots0\rangle$ state. Normally we would work with the representation 
$$|\psi\rangle=\left(\begin{matrix}
    1\\
    0\\
    0\\
    \vdots\\
    0
\end{matrix}\right)$$
with $$|\psi\rangle\in C^{\left(2^{12}\right)}$$
But instead we'll expand the tensor products implicit in $$|00\cdots0\rangle=|0\rangle\otimes|0\rangle\otimes\cdots$$ explicitly by writing out the full tensor. In other words we will instead use
    $$|\psi\rangle\in C^{2\times2\times\cdots\times2}$$
    with
    $$(|\psi\rangle)_{00\cdots0}=1.$$

In [2]:
sites = 12
ψ = zeros((2 for _=1:sites)...)
ψ[(1 for _=1:sites)...] = 1;

Now we convert $|\psi\rangle$ into an MPS:

In [3]:
ψ_mps = mps(ψ);

With this representation, it is much less constly to apply local operators to the state $|\psi\rangle$. Instead of calculating the matrix exponential $e^{-i\hat{H}t}$ using the full hamiltonian $\hat{H}$, we can apply one- or two-local operators at each site in the chain, reinforce the maximum allowed bond dimension, and repeat for each site or pair of sites that is included in $\hat{H}$.

In [4]:
# Define Pauli basis and tensor product shorthand.
# Use t=π/6 for this example.
⊗ = kron
I = [1 0; 0 1]
σ_x = [0 1; 1 0]
σ_y = [0 -1im; 1im 0]
σ_z = [1 0; 0 -1]

t = π / 6;

First, let's calculate the time-evolved state
$$|\psi(t)\rangle = exp\left\{-i\hat{H}t\right\}|\psi\rangle,$$
where the Hamiltonian
$$\hat{H}=\sum_i\sigma_x^{(i)}$$
is a time-independent sum of single-qubit operators. The extremely naive method to calculate $|\psi(t)\rangle$ is to first calculate the matrix exponential, then do a matrix-vector multiplication. Let's do it this way first:

In [5]:
# The not as fast way

# Helper function for calculating H. We only use local terms in this example.
function ising_matrix(sites::Integer, local_only::Bool = false)
        I = [
            1 0
            0 1
        ]
        identity_string = [I for _ = 1:sites]
        local_term = zeros(2^sites, 2^sites)
        interaction_term = zeros(2^sites, 2^sites)

        for i = 1:sites
            pauli_string = copy(identity_string)
            pauli_string[i] = σ_x
            local_term += reduce(kron, pauli_string)
        end

        for i = 1:sites-1
            pauli_string = copy(identity_string)
            pauli_string[i] = σ_z
            pauli_string[i+1] = σ_z
            interaction_term += reduce(kron, pauli_string)
        end
        if local_only
            return local_term
        end
        return interaction_term + local_term
end

# Only use local terms for H:
H = ising_matrix(sites, true)
ψ_vect = zeros(2^sites)
ψ_vect[1] = 1

# Naive method: do the matrix exponential followed by a matrix-vector multiplication.
@time ψ_res_vect = exp(H * t * -1im) * ψ_vect

 39.112074 seconds (6.68 M allocations: 4.726 GiB, 0.80% gc time, 5.08% compilation time)


4096-element Vector{ComplexF64}:
    0.17797851562499956 + 0.0im
                    0.0 - 0.10275594390606246im
                    0.0 - 0.10275594390606206im
   -0.05932617187500035 + 0.0im
                    0.0 - 0.10275594390606241im
   -0.05932617187500032 + 0.0im
   -0.05932617187500048 + 0.0im
                    0.0 + 0.034251981302021194im
                    0.0 - 0.10275594390606213im
  -0.059326171875000305 + 0.0im
  -0.059326171875000305 + 0.0im
                    0.0 + 0.03425198130202117im
  -0.059326171875000416 + 0.0im
                        ⋮
                    0.0 - 0.001268591900074864im
 -0.0007324218750000013 + 0.0im
 -0.0007324218750000011 + 0.0im
                    0.0 + 0.0004228639666916218im
                    0.0 - 0.001268591900074864im
 -0.0007324218750000008 + 0.0im
  -0.000732421875000001 + 0.0im
                    0.0 + 0.00042286396669162214im
  -0.000732421875000001 + 0.0im
                    0.0 + 0.000422863966691622im
                    

Thanks to the `@time` macro, we can see that this calculation takes about 20 seconds and uses a whole lot of memory. Pretty bad, but this is an extremeley naive way of doing things, especially since our initial state is unentangled and the Hamiltonian we're time-evolving under doesn't generate additional entanglement. Using block evolution on a MPS representation of $|\psi\rangle$ effectively takes advantage of this structure. With this method, we only do twelve $2\times2$ matrix-vector multiplications, and since our initial state was unentangled the result is exact. Were our initial state (or system Hamiltonian) more highly entangled (entangling) the MPS representation (block evolution) would do a poorer job approximating the full matrix multiplication.

In [7]:
# The fast way
local_only_ising = Hamiltonian(0 * (σ_z ⊗ σ_z), σ_x)
@time ψ_res = reshape(contract_mps(block_evolve(ψ_mps, local_only_ising, t * -1im)), 2^sites)

  0.005424 seconds (134.92 k allocations: 9.756 MiB)


4096-element Vector{ComplexF64}:
      0.1779785156250009 - 7.705139889271523e-35im
  -4.448564589214635e-35 - 0.10275594390606424im
  -1.689546579027104e-35 - 0.10275594390606424im
    -0.05932617187500025 + 9.754601722097095e-36im
   7.323680981525651e-35 - 0.10275594390606425im
    -0.05932617187500026 - 4.228329186142776e-35im
   -0.059326171875000264 - 1.605902076541324e-35im
  -9.271679961833121e-36 + 0.034251981302021395im
  -4.448564589214636e-35 - 0.10275594390606424im
    -0.05932617187500025 + 2.568379963090506e-35im
    -0.05932617187500025 + 9.754601722097097e-36im
  5.6318219300903436e-36 + 0.03425198130202139im
    -0.05932617187500027 - 4.2283291861427763e-35im
                         ⋮
   9.041581458673635e-37 - 0.0012685919000748663im
  -0.0007324218750000026 - 5.22015948906515e-37im
  -0.0007324218750000027 - 1.982595156223855e-37im
 -1.1446518471398905e-37 + 0.0004228639666916218im
  -5.492055048413126e-37 - 0.001268591900074866im
  -0.0007324218750000024 + 3.17083

To prove the two results are equal:

In [8]:
dot(ψ_res_vect, ψ_res)

1.0000000000000018 - 1.2379560085042535e-34im

Let's switch to a full 1D Ising model. Now
$$\hat{H}=\sum_{i=1}^{N-1}\sigma_z^{(i)}\sigma_z^{(i+1)}+J\sum_{i=1}^N\sigma_x^{(i)}$$
where $J$ tunes the relative strength of the transverse field. For this example we set $J=1.$

Since now $\hat{H}$ generates entanglement, the fidelity of the block-evolved state will not be as high as $t$ grows.

In [9]:
# The not as fast way
t = π/6
# Set H to the full ising matrix as above:
H = ising_matrix(sites, false)
ψ_vect = zeros(2^sites)
ψ_vect[1] = 1

# Naive method: do the matrix exponential followed by a matrix-vector multiplication.
@time ψ_res_vect = exp(H * t * -1im) * ψ_vect

 35.664970 seconds (46 allocations: 4.625 GiB, 0.49% gc time)


4096-element Vector{ComplexF64}:
     0.29784907575844494 - 0.12009697957275987im
     0.03634436868460402 - 0.16293484550907714im
     0.09654847370992732 - 0.09095118660081994im
    -0.07113605814760757 - 0.05259950195585174im
     0.09186977602732573 - 0.09257112598573201im
   -0.010434946441878612 - 0.06826579407664689im
   -0.024114299625164915 - 0.07561913090078207im
   -0.040518920541511086 + 0.02295149606526123im
     0.09215514759497268 - 0.09266111666055488im
   -0.012571664046359515 - 0.0666175975600081im
    0.023814429341693148 - 0.04904671741692144im
    -0.03531296770389746 - 0.00959268034582396im
    -0.02551799874030579 - 0.0732900781298512im
                         ⋮
  -0.0007266283528538789 + 0.0002764548462853698im
 -0.00011164052658427855 + 0.000396427410384179im
   -5.229592898605845e-5 + 0.0004729492134346979im
   0.0002774308701005088 - 2.711314362155868e-5im
 -0.00041153487916395314 + 0.0009021640528471665im
   0.0002647708218863237 + 0.0004410618098450794im
 

In [10]:
ising = Hamiltonian(σ_z ⊗ σ_z, σ_x)
@time ψ_res = reshape(contract_mps(block_evolve(ψ_mps, ising, t * -1im)), 2^sites)

  0.006105 seconds (134.93 k allocations: 9.758 MiB)


4096-element Vector{ComplexF64}:
      0.2636456437307441 - 0.03190778123011212im
     0.13342159770239706 - 0.07554946229865185im
      0.1272377810087296 + 0.009275725040501233im
    0.005355342349062327 - 0.0734607671164807im
     0.12723778100872957 + 0.009275725040501254im
     0.06986968593408303 - 0.023310748261050417im
     0.03761251414157269 - 0.04838575768876558im
   -0.027935530226552815 - 0.02171559516453559im
     0.12723778100872965 + 0.0092757250405013im
     0.06986968593408309 - 0.023310748261050424im
     0.05913012792100387 + 0.01610933701961883im
    0.009300730064743332 - 0.03413879527240859im
     0.03761251414157272 - 0.04838575768876557im
                         ⋮
 -0.00021741980318665746 - 0.00037658214569091667im
 -0.00019949443066899643 - 0.00015241544067346377im
 -0.00020845711692782707 - 1.3444029388245772e-5im
   -7.761913986296907e-6 + 0.00012035277257277423im
  -0.0004312372895557145 + 5.585591613733147e-5im
 -0.00021741980318665748 + 0.000125527381896

In [11]:
dot(ψ_res_vect, ψ_res)

0.21694345086338168 + 0.784587895340711im

The single-pass TEBD state $|\psi\rangle$ is not a very good approximation to the true time-evolved target state $|\phi\rangle$. This is expected since even disregarding the fact that we used a matrix product state, 
    $$e^{-i\left(\sum_{i=1}^NZ_iZ_{i+1}+\sum_{i=1}^NX_i\right)t}\neq\prod_{i=1}^{N-1}e^{-iZ_iZ_{i+1}t}\prod_{i=1}^{N}e^{-iX_it}$$
More succinctly, a single pass of TEBD is essentially a Trotter decomposition for approximating time evolution under $\hat{H}$.

To better approximate the target state $|\phi\rangle$, we can instead do ten passes of block evolution and increase the bond dimension of $|\psi\rangle$, sacrificing memory and runtime:

In [24]:
ψ_mps_high_χ = mps(ψ, 2^(sites ÷ 2))
@time ψ_res_better = reshape(contract_mps(tebd(ψ_mps_high_χ, ising, t * -1im, 10)), 2^sites)
dot(ψ_res_better, ψ_res_vect)

  6.268623 seconds (76.19 M allocations: 9.273 GiB, 3.43% gc time)


0.9944260944838871 - 0.09073247654456926im